In [ ]:
import os
import re
import codecs
import xlrd
import logging
from pprint import pprint

In [ ]:
class LocalizedStringLineParser(object):
    ''' Parses single lines and creates LocalizedString objects from them'''
    def __init__(self):
        # Possible Parsing states indicating what is waited for
        self.ParseStates = {'COMMENT': 1, 'STRING': 2, 'TRAILING_COMMENT': 3, 
                            'STRING_MULTILINE': 4, 'COMMENT_MULTILINE' :5}
        # The parsing state indicates what the last parsed thing was
        self.parse_state = self.ParseStates['COMMENT']
        self.key = None
        self.value = None
        self.comment = None
        self.translatedValue = None

    def parse_line(self, line):
        ''' Parses a single line. Keeps track of the current state and creates
        LocalizedString objects as appropriate

        Keyword arguments:

            line
                The next line to be parsed

        Examples

            >>> parser = LocalizedStringLineParser()
            >>> string = parser.parse_line('    ')
            >>> string

            >>> string = parser.parse_line('/* Comment1 */')
            >>> string

            >>> string = parser.parse_line('    ')
            >>> string

            >>> string = parser.parse_line('"key1" = "value1";')
            >>> string.key
            'key1'
            >>> string.value
            'value1'
            >>> string.comment
            'Comment1'

            >>> string = parser.parse_line('/* Comment2 */')
            >>> string

            >>> string = parser.parse_line('"key2" = "value2";')
            >>> string.key
            'key2'
            >>> string.value
            'value2'
            >>> string.comment
            'Comment2'


            >>> parser = LocalizedStringLineParser()
            >>> string = parser.parse_line('"KEY3" = "VALUE3"; /* Comment3 */')
            >>> string.key
            'KEY3'
            >>> string.value
            'VALUE3'
            >>> string.comment
            'Comment3'



            >>> parser = LocalizedStringLineParser()
            >>> string = parser.parse_line('/* Comment4 */')
            >>> string

            >>> string = parser.parse_line('"KEY4" = "VALUE4')
            >>> string

            >>> string = parser.parse_line('VALUE4_LINE2";')
            >>> string.key
            'KEY4'
            >>> string.value
            'VALUE4\\nVALUE4_LINE2'

            >>> parser = LocalizedStringLineParser()
            >>> string = parser.parse_line('/* Line 1')
            
            >>> string = parser.parse_line(' Line 2')
            
            >>> string = parser.parse_line(' Line 3 */')
            
            >>> string = parser.parse_line('"key" = "value";')
            
            >>> string.key
            'key'
            >>> string.value
            'value'
            >>> string.comment
            'Line 1\\n Line 2\\n Line 3 '
        '''
        if self.parse_state == self.ParseStates['COMMENT']:
            (self.key, self.value, self.comment) = LocalizedString.parse_trailing_comment(line)
            if self.key is not None and self.value is not None and self.comment is not None:
                return self.build_localizedString()
            self.comment = LocalizedString.parse_comment(line)
            if self.comment is not None:
                self.parse_state = self.ParseStates['STRING']
                return None
            # Maybe its a multiline comment
            self.comment_partial = LocalizedString.parse_multiline_comment_start(line)
            if self.comment_partial is not None:
                self.parse_state = self.ParseStates['COMMENT_MULTILINE']
            return None

        elif self.parse_state == self.ParseStates['COMMENT_MULTILINE']:
            comment_end = LocalizedString.parse_multiline_comment_end(line)
            if comment_end is not None:
                self.comment = self.comment_partial + '\n' + comment_end
                self.comment_partial = None
                self.parse_state = self.ParseStates['STRING']
                return None
            # Or its just an intermediate line
            comment_line = LocalizedString.parse_multiline_comment_line(line)
            if comment_line is not None:
                self.comment_partial = self.comment_partial + '\n' + comment_line
            return None

        elif self.parse_state == self.ParseStates['TRAILING_COMMENT']:
            self.comment = LocalizedString.parse_comment(line)
            if self.comment is not None:
                self.parse_state = self.ParseStates['COMMENT']
                return self.build_localizedString()
            return None

        elif self.parse_state == self.ParseStates['STRING']:
            (self.key, self.value) = LocalizedString.parse_localized_pair(
                line
            )
            if self.key is not None and self.value is not None:
                self.parse_state = self.ParseStates['COMMENT']
                return self.build_localizedString()
            # Otherwise, try if the Value is multi-line
            (self.key, self.value_partial) = LocalizedString.parse_multiline_start(
                line
            )
            if self.key is not None and self.value_partial is not None:
                self.parse_state = self.ParseStates['STRING_MULTILINE']
                self.value = None
            return None
        elif self.parse_state == self.ParseStates['STRING_MULTILINE']:
            value_part = LocalizedString.parse_multiline_end(line)
            if value_part is not None:
                self.value = self.value_partial + '\n' + value_part
                self.value_partial = None
                self.parse_state = self.ParseStates['COMMENT']
                return self.build_localizedString()
            value_part = LocalizedString.parse_multiline_line(line)
            if value_part is not None:
                self.value_partial = self.value_partial + '\n' +  value_part
            return None


    def build_localizedString(self):
        localizedString = LocalizedString(
            self.key,
            self.value,
            self.comment
        )
        self.key = None
        self.value = None
        self.comment = None
        return localizedString

class LocalizedString(object):
    ''' A localizes string entry with key, value and comment'''
    COMMENT_EXPR = re.compile(
        # Line start
        '^\w*'
        # Comment
        '/\* (?P<comment>.+) \*/'
        # End of line
        '\w*$'
    )
    COMMENT_MULTILINE_START = re.compile(
        # Line start
        '^\w*'
        # Comment
        '/\* (?P<comment>.+)'
        # End of line
        '\w*$'
    )
    COMMENT_MULTILINE_LINE = re.compile(
        # Line start
        '^'
        # Value
        '(?P<comment>.+)'
        # End of line
        '$'
    )
    COMMENT_MULTILINE_END = re.compile(
        # Line start
        '^'
        # Comment
        '(?P<comment>.+)\*/'
        # End of line
        '\s*$'
    )
    LOCALIZED_STRING_EXPR = re.compile(
        # Line start
        '^'
        # Key
        '"(?P<key>.+)"'
        # Equals
        ' ?= ?'
        # Value
        '"(?P<value>.+)"'
        # Whitespace
        ';'
        # End of line
        '$'
    )
    LOCALIZED_STRING_MULTILINE_START_EXPR = re.compile(
        # Line start
        '^'
        # Key
        '"(?P<key>.+)"'
        # Equals
        ' ?= ?'
        # Value
        '"(?P<value>.+)'
        # End of line
        '$'
    )
    LOCALIZED_STRING_MULTILINE_LINE_EXPR = re.compile(
        # Line start
        '^'
        # Value
        '(?P<value>.+)'
        # End of line
        '$'
    )
    LOCALIZED_STRING_MULTILINE_END_EXPR = re.compile(
        # Line start
        '^'
        # Value
        '(?P<value>.+)"'
        # Whitespace
        ' ?; ?'
        # End of line
        '$'
    )
    LOCALIZED_STRING_TRAILING_COMMENT_EXPR = re.compile(
        # Line start
        '^'
        # Key
        '"(?P<key>.+)"'
        # Equals
        ' ?= ?'
        # Value
        '"(?P<value>.+)"'
        # Whitespace
        ' ?; ?'
        # Comment
        '/\* (?P<comment>.+) \*/'
        # End of line
        '$'

    )

    @classmethod
    def parse_multiline_start(cls, line):
        ''' Parse the beginning of a multi-line entry, "KEY"="VALUE_LINE1

        Keyword arguments:

            line
                The line to be parsed

        Returns
            ``tuple`` with key, value and comment
            ``None`` when the line was no comment

        Examples

            >>> line = '"key" = "value4'
            >>> LocalizedString.parse_multiline_start(line)
            ('key', 'value4')

        '''
        result = cls.LOCALIZED_STRING_MULTILINE_START_EXPR.match(line)
        if result is not None:
            return (result.group('key'),
                    result.group('value'))
        else:
            return (None, None)

    @classmethod
    def parse_multiline_line(cls, line):
        ''' Parse an intermediate line of a multi-line entry, only value

        Keyword arguments:

            line
                The line to be parsed

        Returns
            ``String`` with the value
            ``None`` when the line was no comment

        Examples

            >>> line = 'value4, maybe something else'
            >>> LocalizedString.parse_multiline_line(line)
            'value4, maybe something else'
        '''
        result = cls.LOCALIZED_STRING_MULTILINE_LINE_EXPR.match(line)
        if result is not None:
            return result.group('value')
        return None


    @classmethod
    def parse_multiline_end(cls, line):
        ''' Parse an end line of a multi-line entry, only value

        Keyword arguments:

            line
                The line to be parsed

        Returns
            ``String`` the value 
            ``None`` when the line was no comment

        Examples

            >>> line = 'value4, maybe something else";'
            >>> LocalizedString.parse_multiline_end(line)
            'value4, maybe something else'
        '''
        result = cls.LOCALIZED_STRING_MULTILINE_END_EXPR.match(line)
        if result is not None:
            return result.group('value')
        return None


    @classmethod
    def parse_trailing_comment(cls, line):
        '''Extract the content of a line with a trailing comment.

        Keyword arguments:

            line
                The line to be parsed

        Returns
            ``tuple`` with key, value and comment
            ``None`` when the line was no comment

        Examples

            >>> line = '"key3" = "value3";/* Bla */'
            >>> LocalizedString.parse_trailing_comment(line)
            ('key3', 'value3', 'Bla')
        '''
        result = cls.LOCALIZED_STRING_TRAILING_COMMENT_EXPR.match(line)
        if result is not None:
            return (
                result.group('key'),
                result.group('value'),
                result.group('comment')
            )
        else:
            return (None, None, None)

    @classmethod
    def parse_multiline_comment_start(cls, line):
        '''
        Example:

            >>> LocalizedString.parse_multiline_comment_start('/* Hello ')
            'Hello '
        '''
        result = cls.COMMENT_MULTILINE_START.match(line)
        if result is not None:
            return result.group('comment')
        else:
            return None


    @classmethod
    def parse_multiline_comment_line(cls, line):
        '''
        Example:

            >>> LocalizedString.parse_multiline_comment_line(' Line ')
            ' Line '
        '''
        result = cls.COMMENT_MULTILINE_LINE.match(line)
        if result is not None:
            return result.group('comment')
        else:
            return None


    @classmethod
    def parse_multiline_comment_end(cls, line):
        '''
        Example:

            >>> LocalizedString.parse_multiline_comment_end(' End */ ')
            ' End '
        '''
        result = cls.COMMENT_MULTILINE_END.match(line)
        if result is not None:
            return result.group('comment')
        else:
            return None

    @classmethod
    def parse_comment(cls, line):
        '''Extract the content of a comment line from a line.

        Keyword arguments:

            line
                The line to be parsed

        Returns
            ``string`` with the Comment or
            ``None`` when the line was no comment

        Examples

            >>> LocalizedString.parse_comment('This line is no comment')
            >>> LocalizedString.parse_comment('')
            >>> LocalizedString.parse_comment('/* Comment */')
            'Comment'
        '''
        result = cls.COMMENT_EXPR.match(line)
        if result is not None:
            return result.group('comment')
        else:
            return None

    @classmethod
    def parse_localized_pair(cls, line):
        '''Extract the content of a key/value pair from a line.

        Keyword arguments:

            line
                The line to be parsed

        Returns
            ``tupple`` with key and value as strings
            ``tupple`` (None, None) when the line was no match

        Examples

            >>> LocalizedString.parse_localized_pair('Some Line')
            (None, None)
            >>> LocalizedString.parse_localized_pair('/* Comment */')
            (None, None)
            >>> LocalizedString.parse_localized_pair('"key1" = "value1";')
            ('key1', 'value1')
        '''
        result = cls.LOCALIZED_STRING_EXPR.match(line)
        if result is not None:
            return (
                result.group('key'),
                result.group('value')
            )
        else:
            return (None, None)

    def __eq__(self, other):
        '''Tests Equality of two LocalizedStrings

        >>> s1 = LocalizedString('key1', 'value1', 'comment1')
        >>> s2 = LocalizedString('key1', 'value1', 'comment1')
        >>> s3 = LocalizedString('key1', 'value2', 'comment1')
        >>> s4 = LocalizedString('key1', 'value1', 'comment2')
        >>> s5 = LocalizedString('key1', 'value2', 'comment2')
        >>> s1 == s2
        True
        >>> s1 == s3
        False
        >>> s1 == s4
        False
        >>> s1 == s5
        False
        '''
        if isinstance(other, LocalizedString):
            return (self.key == other.key and self.value == other.value and
                    self.comment == other.comment)
        else:
            return NotImplemented

    def __neq__(self, other):
        result = self.__eq__(other)
        if(result is NotImplemented):
            return result
        return not result

    def __init__(self, key, value=None, comment=None):
        super(LocalizedString, self).__init__()
        self.key = key
        self.value = value
        self.comment = comment

    def is_raw(self):
        '''
        Return True if the localized string has not been translated.

        Examples
            >>> l1 = LocalizedString('key1', 'valye1', 'comment1')
            >>> l1.is_raw()
            False
            >>> l2 = LocalizedString('key2', 'key2', 'comment2')
            >>> l2.is_raw()
            True
        '''
        return self.value == self.key

    def __str__(self):
        if self.comment:
            return '/* %s */\n"%s" = "%s";\n' % (
                self.comment, self.key or '', self.value or '',
            )
        else:
            return '"%s" = "%s";\n' % (self.key or '', self.value or '')
        
    def __unicode__(self):
        if self.comment:
            return u'/* %s */\n"%s" = "%s";\n' % (
                self.comment, self.key or u'', self.value or u'',
            )
        else:
            return u'"%s" = "%s";\n' % (self.key or u'', self.value or u'')

In [ ]:


XCODE_PATH = "/Users/alfredreynold/Documents/"

# Create Logger
logging.basicConfig(
    format='%(message)s',
    level="verbose"
)
def parse_file(file_path, encoding='utf16'):
    ''' Parses a file and creates a dictionary containing all LocalizedStrings
        elements in the file

        Keyword arguments:

            file_path
                path to the file that should be parsed

            encoding
                encoding of the file

        Returns:    ``dict``
    '''
    
    with codecs.open(file_path, mode='r', encoding=encoding) as file_contents:
        logging.debug("Parsing File: {}".format(file_path))
        parser = LocalizedStringLineParser()
        localized_strings = {}
        try:
            for line in file_contents:
                localized_string = parser.parse_line(line)
                if localized_string is not None:
                    localized_strings[localized_string.key] = localized_string
        except UnicodeError:
            logging.debug("Failed to open file as UTF16, Trying UTF8")
            file_contents = codecs.open(file_path, mode='r', encoding='utf8')
            for line in file_contents:
                localized_string = parser.parse_line(line)
                if localized_string is not None:
                    localized_strings[localized_string.key] = localized_string
    return localized_strings



def readStringFiles():
    xibFileNames = []    
    LPROJ_PATH = "/Users/alfredreynold/Documents/proj/Indian/en-in.lproj/" 
    allStringFiles={}
    for files in os.listdir(LPROJ_PATH):
        if files.endswith(".strings") and os.path.exists(LPROJ_PATH+files.split('.')[0]+'.xib'):
            xibFileNames.append(files)
            dicFile={}            
            allStringFiles[files]=parse_file(LPROJ_PATH+files)
        if files.endswith(".strings") :
            if os.path.exists(LPROJ_PATH+files.split('.')[0]+'.xib'):
                xibFileNames.append(files)
                dicFile={}            
                allStringFiles[files]=parse_file(LPROJ_PATH+files)
            else :
                print "No xib for : ",files
        if files == "Localizable.strings":
            allStringFiles[files]=parse_file(LPROJ_PATH+files)
    return allStringFiles
            

def readTranslatedXL():
    XL_PATH= "/Users/alfredreynold/Downloads/Tamil-AllUntranslatedStrings2-20.05.2016.xls"
    bk = xlrd.open_workbook(XL_PATH)
    tdic={}
    for sht in bk.sheets():
        print sht.name
        for row in range(1,sht.nrows):
            tdic[str(sht.cell_value(row,1).encode('utf-8'))] = str(sht.cell_value(row,3).encode('utf-8'))
    return tdic
        
def writeTranslation(fileName,translatedValue):
    LPROJ_PATH = "/Users/alfredreynold/Documents/proj/Indian/en-in.lproj/" 
    stringFile = codecs.open(LPROJ_PATH+fileName,'w','utf-8') 
    for v in translatedValue:
        temp_key = unicode(v)
        stringFile.write(temp_key+u"\n") 
    stringFile.close() 
    
def run(destlang):
    translatedDic = readTranslatedXL()
    stringFILES = readStringFiles()
    
    for fn,dic in stringFILES.iteritems():
        fileName=fn
        print "FILENAME: "+fileName
        stringEntry = []
        for k,v in dic.iteritems():
            print v.value
            if v.value in translatedDic:
                print "Found : ",k,v.value
                v.value = translatedDic[v.value].decode('utf-8')
            stringEntry.append(v)
            
                #Each file 'fn' replace it's value with v.translatedValue 
        writeTranslation(fileName,stringEntry)
    
    
run("en-in")

In [ ]:
translatedDic = readTranslatedXL()

In [ ]:
print translatedDic["Retain current vocabulary?"]

In [ ]:
from subprocess import call
LPROJ_PATH = "/Users/alfredreynold/Documents/Indian/en-in.lproj/" 
allStringFiles={}
for files in os.listdir(LPROJ_PATH):
    stringsFile = LPROJ_PATH+files
    xibFile = LPROJ_PATH+files.split('.')[0]+'.xib'
    if files.endswith(".strings") and os.path.exists(LPROJ_PATH+files.split('.')[0]+'.xib'):
# ibtool --strings-file da.lproj/CustomizationView.strings --write da.lproj/CustomizationView.xib da.lproj/CustomizationView.xib
        exexuteString = "ibtool --strings-file "+stringsFile+" --write "+xibFile +" "+xibFile
        print exexuteString
        os.system(exexuteString)